In [1]:
import glob,sys
from pathlib import Path
pinkRig_path= glob.glob(r'C:\Users\*\Documents\Github\PinkRigs')
pinkRig_path = Path(pinkRig_path[0])
sys.path.insert(0, (pinkRig_path.__str__()))
kilofolder = __import__("Analysis.+kilo.python_.convert_to_ibl_format") # this is necessary because some wise people decided to write a package with python and matlab in it. #wisdom.
kilo = getattr(kilofolder, "+kilo")
add_anat_to_ibl_format = kilo.python_.convert_to_ibl_format.add_anat_to_ibl_format

# Path to raw ephys data
#ephys_path = Path(r'Z:\AV014\2022-07-07\ephys\AV014_2022-07-07_NatImagesSparseNoise_g0\AV014_2022-07-07_NatImagesSparseNoise_g0_imec0')
     
#add_anat_to_ibl_format(ephys_path,ks_folder='pyKS',recompute=True)

In [4]:
# add a func to loop over SC recordings (selected somehow by Flora)
import json
import pandas as pd 
from Admin.csv_pyhandlers import get_server_location
from Analysis.helpers.queryExp import load_data

prPath = get_server_location()
session_list = pd.read_csv(prPath / 'Helpers/sc_selected_recordings.csv')


for _,r in session_list.iterrows():
    # read corresponding ephys files 
    corresponding_ephys_json= list((Path(r.expFolder)).glob('ONE_preproc\probe*\_av_rawephys.path*.json'))

    for rec_path in corresponding_ephys_json: 
            # open json to get the actual path
            rec = open(rec_path,)
            rec = json.load(rec)
            rec = Path(rec)
            print(rec.parents[2])
            s=add_anat_to_ibl_format(rec.parents[2],recompute=False)


\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\ephys\AV025_2022-11-07_SparseNoise_Part1_g0\AV025_2022-11-07_SparseNoise_Part1_g0_imec0
converting data to IBL format ...


c:\Users\Experiment\anaconda3\envs\iblenv\lib\site-packages\phylib\io\model.py:1053: RuntimeWarning: invalid value encountered in true_divide
  spikes_depths[ispi] = (np.sum(np.transpose(ypos * features) /
Extracting waveforms: 100%|██████████| 655.4470333333334/655.4470333333334 [00:36<00:00, 17.75it/s]
Converting to ALF:  84%|████████▍ | 105/125 [00:41<00:09,  2.08it/s]c:\Users\Experiment\anaconda3\envs\iblenv\lib\site-packages\phylib\io\model.py:1053: RuntimeWarning: invalid value encountered in true_divide
  spikes_depths[ispi] = (np.sum(np.transpose(ypos * features) /
Converting to ALF: 145it [00:47,  3.03it/s]                         
2023-01-13 00:11:49.867 INFO     [extract_files.py:82] Computing QC for \\zaru.cortexlab.net\Subjects\AV025\2022-11-07\ephys\AV025_2022-11-07_SparseNoise_Part1_g0\AV025_2022-11-07_SparseNoise_Part1_g0_imec0\AV025_2022-11-07_SparseNoise_Part1_g0_t0.imec0.ap.bin


the probe/shank IDs do not match.████████████████████████████████████████████████████████████████████-| 99.3% 
\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\ephys\AV025_2022-11-07_SparseNoise_Part1_g0\AV025_2022-11-07_SparseNoise_Part1_g0_imec1
converting data to IBL format ...


Extracting waveforms: 100%|██████████| 655.4470333333334/655.4470333333334 [00:30<00:00, 21.20it/s]
Converting to ALF: 145it [00:39,  3.66it/s]                         
2023-01-13 00:18:15.819 INFO     [extract_files.py:82] Computing QC for \\zaru.cortexlab.net\Subjects\AV025\2022-11-07\ephys\AV025_2022-11-07_SparseNoise_Part1_g0\AV025_2022-11-07_SparseNoise_Part1_g0_imec1\AV025_2022-11-07_SparseNoise_Part1_g0_t0.imec1.ap.bin


the probe/shank IDs do not match.████████████████████████████████████████████████████████████████████-| 99.3% 
\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\ephys\AV025_2022-11-07_SparseNoise_Part2_g0\AV025_2022-11-07_SparseNoise_Part2_g0_imec0
converting data to IBL format ...


...

Extracting waveforms: 100%|██████████| 640.7795333333333/640.7795333333333 [00:06<00:00, 97.32it/s]
Converting to ALF: 145it [00:11, 12.29it/s]                         
2023-01-13 08:37:23.826 INFO     [extract_files.py:82] Computing QC for \\zaru.cortexlab.net\Subjects\AV025\2022-11-14\ephys\AV025_2022-11-14_SparseNoise_Part2_g0\AV025_2022-11-14_SparseNoise_Part2_g0_imec1\AV025_2022-11-14_SparseNoise_Part2_g0_t0.imec1.ap.cbin


 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
the probe/shank IDs do not match.
\\zaru.cortexlab.net\Subjects\AV025\2022-11-14\ephys\AV025_2022-11-14_SparseNoise_Part3_g0\AV025_2022-11-14_SparseNoise_Part3_g0_imec0
converting data to IBL format ...


Extracting waveforms: 100%|██████████| 751.0699/751.0699 [00:06<00:00, 115.16it/s]
Converting to ALF: 145it [00:11, 13.01it/s]                         
2023-01-13 08:43:25.628 INFO     [extract_files.py:82] Computing QC for \\zaru.cortexlab.net\Subjects\AV025\2022-11-14\ephys\AV025_2022-11-14_SparseNoise_Part3_g0\AV025_2022-11-14_SparseNoise_Part3_g0_imec0\AV025_2022-11-14_SparseNoise_Part3_g0_t0.imec0.ap.cbin


the probe/shank IDs do not match.████████████████████████████████████████████████████████████████████-| 99.4% 
\\zaru.cortexlab.net\Subjects\AV025\2022-11-14\ephys\AV025_2022-11-14_SparseNoise_Part3_g0\AV025_2022-11-14_SparseNoise_Part3_g0_imec1
converting data to IBL format ...


Extracting waveforms: 100%|██████████| 751.0675333333334/751.0675333333334 [00:04<00:00, 156.12it/s]
Converting to ALF: 145it [00:07, 20.46it/s]                         
2023-01-13 08:50:24.9 INFO     [extract_files.py:82] Computing QC for \\zaru.cortexlab.net\Subjects\AV025\2022-11-14\ephys\AV025_2022-11-14_SparseNoise_Part3_g0\AV025_2022-11-14_SparseNoise_Part3_g0_imec1\AV025_2022-11-14_SparseNoise_Part3_g0_t0.imec1.ap.cbin
